In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold, GroupKFold


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
submission = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')

EDA and feature selection is done in my other [notebook](https://www.kaggle.com/code/robertturro/tps-april-2022-eda)

In [ ]:
train.shape

In [ ]:
groups = train["sequence"]
train = train.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
test = test.drop(["sequence", "subject", "step"], inplace=False, axis=1).values

sc = StandardScaler()

sc.fit(train)
train = sc.transform(train)
test = sc.transform(test)

labels = labels["state"]
#int(len(train)/60)
train = train.reshape(-1, 60, 13)
test = test.reshape(-1, 60, 13)

In [ ]:
x_input = Input(shape=(train.shape[-2:]))
    
x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
x2 = Bidirectional(LSTM(units=256, return_sequences=True))(x1)
z1 = Bidirectional(GRU(units=256, return_sequences=True))(x1)
    
c = Concatenate(axis=2)([x2, z1])
    
x3 = Bidirectional(LSTM(units=128, return_sequences=True))(c)
    
x4 = GlobalMaxPooling1D()(x3)
x5 = Dense(units=128, activation='selu')(x4)
x_output = Dense(1, activation='sigmoid')(x5)

model = Model(inputs=x_input, outputs=x_output, name='lstm_model')
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC()])
model.fit(train,labels, validation_split=0.3, epochs=75, batch_size=256,callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

In [ ]:
test_preds = model.predict(test)

In [ ]:
submission["state"] = test_preds
submission.to_csv("submission.csv", index=False)
submission

# References

* [Tps April Tensorflow Bi-LSTM](https://www.kaggle.com/code/hamzaghanmi/tps-april-tensorflow-bi-lstm)
* [LSTM Baseline](https://www.kaggle.com/code/ryanbarretto/lstm-baseline)